<a href="https://colab.research.google.com/github/muhammad-sodo/Projects/blob/main/Project_1_Streamlit_Web_App_With_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# streamlit_app.py

import streamlit as st
import pandas as pd
import requests
import pydeck as pdk

st.set_page_config(page_title="Toronto BikeShare Live Map", layout="wide")

st.title("🚲 Toronto BikeShare - Live Bike Station Map")

# GTFS JSON feeds
stations_url = "https://tor.publicbikesystem.net/ube/gbfs/en/station_information.json"
status_url = "https://tor.publicbikesystem.net/ube/gbfs/en/station_status.json"

@st.cache_data(ttl=60)  # Updates data every 60 seconds
def load_data():
    stations = requests.get(stations_url).json()
    status = requests.get(status_url).json()

    stations_df = pd.DataFrame(stations["data"]["stations"])
    status_df = pd.DataFrame(status["data"]["stations"])

    merged_df = pd.merge(stations_df, status_df, on="station_id")
    return merged_df

df = load_data()

# Interactive filter
min_bikes = st.slider("Minimum number of available bikes:", 0, 20, 1)

filtered_df = df[df["num_bikes_available"] >= min_bikes]

# Display map
st.subheader("📍 Station Map")
st.pydeck_chart(pdk.Deck(
    map_style='mapbox://styles/mapbox/light-v9',
    initial_view_state=pdk.ViewState(
        latitude=43.651070,
        longitude=-79.347015,
        zoom=12,
        pitch=50,
    ),
    layers=[
        pdk.Layer(
            'ScatterplotLayer',
            data=filtered_df,
            get_position='[lon, lat]',
            get_color='[200, 30, 0, 160]',
            get_radius=100,
        ),
        pdk.Layer(
            "TextLayer",
            data=filtered_df,
            get_position='[lon, lat]',
            get_text='name',
            get_size=12,
            get_color=[0, 0, 0],
            get_angle=0,
            get_alignment_baseline="'bottom'",
        )
    ],
))

# Display data table
st.subheader("📊 Station Details")
st.dataframe(filtered_df[["name", "num_bikes_available", "num_docks_available"]])

ModuleNotFoundError: No module named 'streamlit'